<div style="display:block">
    <div style="width: 20%; display: inline-block; text-align: left;">
        <div class="crop">
            <img src="http://networthcorp.net/NetworthCorp_LogoUsage_R1-01.jpg" style="height:75px; margin-left:0px" />
        </div>
    </div>
    <div style="width: 59%; display: inline-block">
            <h1  style="text-align: center">Pairwise_Matching</h1><br>
        <div style="width: 90%; text-align: center; display: inline-block;"><i>Author:</i> <strong>NetworthCorp</strong> </div>
    </div>
    <div style="width: 20%; text-align: right; display: inline-block;">
        <div style="width: 100%; text-align: left; display: inline-block;">
            <i>Created: </i>
            <time datetime="Enter Date" pubdate>June, 2018</time>
        </div>
    </div>
</div>

## || Functionality:

__This Function finds the similarity score across all records in a dataframe.__

## || Input Parameters:

* __source:__ It takes the Path or Name of the File.
* __cell_pair_agg:__ It Determines how cell pair aggregation is done. It can take:
   * __avg:__ Average of the all the similarity function.
   * __wtavg:__ Weighted Average of all the similarity function.
   * __best:__ Best similarity score in all the similarity function.
* __row_pair_agg:__ It Determines how row pair aggregation is done. It can take:
   * __avg:__ Average of the all the similarity function.
   * __wtavg:__ Weighted Average of all the similarity function.
<hr>
* __config_files_test:__ It stores the configuration of the function: 
   * __coltype_func:__ It stores the similarity function used in different column types.
   * __colname_func:__ It stores the similarity function used in different columns according to the user.
   * __column_weight:__ It is the weights of columns for finding weighted row pair aggregation.
   * __cell_pair_weight:__ It is the weights of functions in columns for finding weighted cell pair aggregation.

## || Return:

__It returns a matrix of rows x rows with similarity scores according to the index.__

## || Code:

In [202]:
import similarity_method_pairwise as sm
import numpy as np
import pandas as pd
from col_type import check_inputdata_attributes_column_type as column_type

from config_files_test import column_weight as w
from config_files_test import coltype_func as cf
from config_files_test import colname_func as cnf
from config_files_test import cell_pair_weight as cpw

In [1]:
def pairwise_matching(source,cell_pair_agg='avg',row_pair_agg='avg'):
    
    df=pd.read_csv(source)#.dropna()
    col_type=column_type(source)
    n=len(df)
    pair=pd.DataFrame(columns = ['i','j'])

    for index, column in col_type.iterrows():
        
        for i in range(n):
            
            for j in range(i,n):
                
                if column['Name'] in cnf:
                    temp=[]
                    for value in cnf.get(column['Name']):
                        if value=='numeric_att':
                            dmax=(max(df[column['Name']])-min(df[column['Name']]))
                            score=getattr(sm,value)(df.iloc[i][column['Name']],df.iloc[j][column['Name']],dmax)
                            temp.append(score)
                        else:
                            score=getattr(sm,value)(df.iloc[i][column['Name']],df.iloc[j][column['Name']])
                            temp.append(score)
                        
                    if cell_pair_agg=='wtavg':
                        if column['Name'] in cpw:
                            wt=cpw.get(column['Name'])
                            cell_wtavg= np.average(temp,weights=wt)
                            pair=pair.append({'i': int(i),'j':int(j),column['Name']:cell_wtavg},ignore_index=True)
                        else:
                            avg= np.mean(temp)
                            pair=pair.append({'i': int(i),'j':int(j),column['Name']:avg},ignore_index=True)
                            
                    elif cell_pair_agg=='avg':
                        avg= np.mean(temp)
                        pair=pair.append({'i': int(i),'j':int(j),column['Name']:avg},ignore_index=True)
                        
                    elif cell_pair_agg=='best':
                        best= np.max(temp)
                        pair=pair.append({'i': int(i),'j':int(j),column['Name']:best},ignore_index=True)
                
                elif column['Type_col']=='Constant':
                #Check for user input:
                    temp=[]
                    if ('boolean' in cf.get('Constant'))==True:
                        score1=sm.boolean(df.iloc[i][column['Name']],df.iloc[j][column['Name']])
                        temp.append(score1)
                        
                    if ('direct_score' in cf.get('Constant'))==True:    
                        score2=1.0
                        temp.append(score2)
                #Aggregation:
                    if cell_pair_agg=='wtavg':
                        if column['Name'] in cpw:
                            wt=cpw.get(column['Name'])
                            cell_wtavg= np.average(temp,weights=wt)
                            pair=pair.append({'i': int(i),'j':int(j),column['Name']:cell_wtavg},ignore_index=True)
                        else:
                            avg= np.mean(temp)
                            pair=pair.append({'i': int(i),'j':int(j),column['Name']:avg},ignore_index=True)
                            
                    elif cell_pair_agg=='avg':
                        avg= np.mean(temp)
                        pair=pair.append({'i': int(i),'j':int(j),column['Name']:avg},ignore_index=True)
                        
                    elif cell_pair_agg=='best':
                        best= np.max(temp)
                        pair=pair.append({'i': int(i),'j':int(j),column['Name']:best},ignore_index=True)

                elif column['Type_col']=='Boolean':
                #Check for user input:
                    temp=[]
                    if ('boolean' in cf.get('Boolean'))==True:
                        score1=sm.boolean(df.iloc[i][column['Name']],df.iloc[j][column['Name']])
                        temp.append(score1)
                #Aggregation:    
                    avg= np.mean(temp)
                    pair=pair.append({'i': int(i),'j':int(j),column['Name']:avg},ignore_index=True)

                elif column['Type_col']=='Numeric With Length=1':
                #Check for user input:
                    temp=[]
                    if ('numeric_att' in cf.get('Numeric With Length=1'))==True:
                        dmax=(max(df[column['Name']])-min(df[column['Name']]))
                        score1=sm.numeric_att(df.iloc[i][column['Name']],df.iloc[j][column['Name']],dmax)
                        temp.append(score1)
                        
                    if ('levenshtein' in cf.get('Numeric With Length=1'))==True:
                        score2=sm.levenshtein(df.iloc[i][column['Name']],df.iloc[j][column['Name']])
                        temp.append(score2)
                    
                    if ('boolean' in cf.get('Numeric With Length=1'))==True:
                        score3=sm.boolean(df.iloc[i][column['Name']],df.iloc[j][column['Name']])
                        temp.append(score3)
                #Aggregation:
                    if cell_pair_agg=='wtavg':
                        if column['Name'] in cpw:
                            wt=cpw.get(column['Name'])
                            cell_wtavg= np.average(temp,weights=wt)
                            pair=pair.append({'i': int(i),'j':int(j),column['Name']:cell_wtavg},ignore_index=True)
                        else:
                            avg= np.mean(temp)
                            pair=pair.append({'i': int(i),'j':int(j),column['Name']:avg},ignore_index=True)
                            
                    elif cell_pair_agg=='avg':
                        avg= np.mean(temp)
                        pair=pair.append({'i': int(i),'j':int(j),column['Name']:avg},ignore_index=True)
                        
                    elif cell_pair_agg=='best':
                        best= np.max(temp)
                        pair=pair.append({'i': int(i),'j':int(j),column['Name']:best},ignore_index=True)

                elif column['Type_col']=='Numeric Continuous':
                #Check for user input:
                    temp=[]
                    if ('numeric_att' in cf.get('Numeric Continuous'))==True:
                        dmax=(max(df[column['Name']])-min(df[column['Name']]))
                        score1=sm.numeric_att(df.iloc[i][column['Name']],df.iloc[j][column['Name']],dmax)
                        temp.append(score1)
                        
                    if ('d_levenshtein' in cf.get('Numeric Continuous'))==True:
                        score2=sm.d_levenshtein(df.iloc[i][column['Name']],df.iloc[j][column['Name']])
                        temp.append(score2)
                    
                    if ('jaro_winkler' in cf.get('Numeric Continuous'))==True:
                        score3=sm.jaro_winkler(df.iloc[i][column['Name']],df.iloc[j][column['Name']])
                        temp.append(score3)
                        
                    if ('jaccard' in cf.get('Numeric Continuous'))==True:
                        score4=sm.jaccard(df.iloc[i][column['Name']],df.iloc[j][column['Name']])
                        temp.append(score4)
                    
                #Aggregation:
                    if cell_pair_agg=='wtavg':
                        if column['Name'] in cpw:
                            wt=cpw.get(column['Name'])
                            cell_wtavg= np.average(temp,weights=wt)
                            pair=pair.append({'i': int(i),'j':int(j),column['Name']:cell_wtavg},ignore_index=True)
                        else:
                            avg= np.mean(temp)
                            pair=pair.append({'i': int(i),'j':int(j),column['Name']:avg},ignore_index=True)
                            
                    elif cell_pair_agg=='avg':
                        avg= np.mean(temp)
                        pair=pair.append({'i': int(i),'j':int(j),column['Name']:avg},ignore_index=True)
                        
                    elif cell_pair_agg=='best':
                        best= np.max(temp)
                        pair=pair.append({'i': int(i),'j':int(j),column['Name']:best},ignore_index=True)
                        
                elif column['Type_col']=='Uniformly Spaced':
                #Check for user input:
                    temp=[]
                    if ('numeric_att' in cf.get('Uniformly Spaced'))==True:
                        dmax=(max(df[column['Name']])-min(df[column['Name']]))
                        score1=sm.numeric_att(df.iloc[i][column['Name']],df.iloc[j][column['Name']],dmax)
                        temp.append(score1)
                        
                    if ('d_levenshtein' in cf.get('Uniformly Spaced'))==True:
                        score2=sm.d_levenshtein(df.iloc[i][column['Name']],df.iloc[j][column['Name']])
                        temp.append(score2)
                    
                    if ('jaro_winkler' in cf.get('Uniformly Spaced'))==True:
                        score3=sm.jaro_winkler(df.iloc[i][column['Name']],df.iloc[j][column['Name']])
                        temp.append(score3)
                        
                    if ('jaccard' in cf.get('Uniformly Spaced'))==True:
                        score4=sm.jaccard(df.iloc[i][column['Name']],df.iloc[j][column['Name']])
                        temp.append(score4)
                    
                #Aggregation:
                    if cell_pair_agg=='wtavg':
                        if column['Name'] in cpw:
                            wt=cpw.get(column['Name'])
                            cell_wtavg= np.average(temp,weights=wt)
                            pair=pair.append({'i': int(i),'j':int(j),column['Name']:cell_wtavg},ignore_index=True)
                        else:
                            avg= np.mean(temp)
                            pair=pair.append({'i': int(i),'j':int(j),column['Name']:avg},ignore_index=True)
                            
                    elif cell_pair_agg=='avg':
                        avg= np.mean(temp)
                        pair=pair.append({'i': int(i),'j':int(j),column['Name']:avg},ignore_index=True)
                        
                    elif cell_pair_agg=='best':
                        best= np.max(temp)
                        pair=pair.append({'i': int(i),'j':int(j),column['Name']:best},ignore_index=True)

                elif column['Type_col']=='Non-Uniformly Spaced':
                #Check for user input:
                    temp=[]
                    if ('d_levenshtein' in cf.get('Non-Uniformly Spaced'))==True:
                        score2=sm.d_levenshtein(df.iloc[i][column['Name']],df.iloc[j][column['Name']])
                        temp.append(score2)
                    
                    if ('jaro_winkler' in cf.get('Non-Uniformly Spaced'))==True:
                        score3=sm.jaro_winkler(df.iloc[i][column['Name']],df.iloc[j][column['Name']])
                        temp.append(score3)
                        
                    if ('jaccard' in cf.get('Non-Uniformly Spaced'))==True:
                        score4=sm.jaccard(df.iloc[i][column['Name']],df.iloc[j][column['Name']])
                        temp.append(score4)
                    
                #Aggregation:
                    if cell_pair_agg=='wtavg':
                        if column['Name'] in cpw:
                            wt=cpw.get(column['Name'])
                            cell_wtavg= np.average(temp,weights=wt)
                            pair=pair.append({'i': int(i),'j':int(j),column['Name']:cell_wtavg},ignore_index=True)
                        else:
                            avg= np.mean(temp)
                            pair=pair.append({'i': int(i),'j':int(j),column['Name']:avg},ignore_index=True)
                            
                    elif cell_pair_agg=='avg':
                        avg= np.mean(temp)
                        pair=pair.append({'i': int(i),'j':int(j),column['Name']:avg},ignore_index=True)
                        
                    elif cell_pair_agg=='best':
                        best= np.max(temp)
                        pair=pair.append({'i': int(i),'j':int(j),column['Name']:best},ignore_index=True)


                elif column['Type_col']=='Uniformly Distributed':
                #Check for user input:
                    temp=[]
                    if ('numeric_att' in cf.get('Uniformly Distributed'))==True:
                        dmax=(max(df[column['Name']])-min(df[column['Name']]))
                        score1=sm.numeric_att(df.iloc[i][column['Name']],df.iloc[j][column['Name']],dmax)
                        temp.append(score1)
                        
                    if ('d_levenshtein' in cf.get('Uniformly Distributed'))==True:
                        score2=sm.d_levenshtein(df.iloc[i][column['Name']],df.iloc[j][column['Name']])
                        temp.append(score2)
                    
                    if ('jaro_winkler' in cf.get('Uniformly Distributed'))==True:
                        score3=sm.jaro_winkler(df.iloc[i][column['Name']],df.iloc[j][column['Name']])
                        temp.append(score3)
                        
                    if ('jaccard' in cf.get('Uniformly Distributed'))==True:
                        score4=sm.jaccard(df.iloc[i][column['Name']],df.iloc[j][column['Name']])
                        temp.append(score4)
                    
                #Aggregation:
                    if cell_pair_agg=='wtavg':
                        if column['Name'] in cpw:
                            wt=cpw.get(column['Name'])
                            cell_wtavg= np.average(temp,weights=wt)
                            pair=pair.append({'i': int(i),'j':int(j),column['Name']:cell_wtavg},ignore_index=True)
                        else:
                            avg= np.mean(temp)
                            pair=pair.append({'i': int(i),'j':int(j),column['Name']:avg},ignore_index=True)
                            
                    elif cell_pair_agg=='avg':
                        avg= np.mean(temp)
                        pair=pair.append({'i': int(i),'j':int(j),column['Name']:avg},ignore_index=True)
                        
                    elif cell_pair_agg=='best':
                        best= np.max(temp)
                        pair=pair.append({'i': int(i),'j':int(j),column['Name']:best},ignore_index=True)


                elif column['Type_col']=='Date':
                #Check for user input:    
                    temp=[]
                    if ('date' in cf.get('Uniformly Distributed'))==True:
                        score1=sm.date(df.iloc[i][column['Name']],df.iloc[j][column['Name']])
                        temp.append(score1)
                    
                    if ('levenshtein' in cf.get('Uniformly Distributed'))==True:
                        score2=sm.levenshtein(df.iloc[i][column['Name']],df.iloc[j][column['Name']])
                        temp.append(score2)
                        
                    if ('d_levenshtein' in cf.get('Uniformly Distributed'))==True:
                        score3=sm.d_levenshtein(df.iloc[i][column['Name']],df.iloc[j][column['Name']])
                        temp.append(score3)
                    
                #Aggregation:
                    if cell_pair_agg=='wtavg':
                        if column['Name'] in cpw:
                            wt=cpw.get(column['Name'])
                            cell_wtavg= np.average(temp,weights=wt)
                            pair=pair.append({'i': int(i),'j':int(j),column['Name']:cell_wtavg},ignore_index=True)
                        else:
                            avg= np.mean(temp)
                            pair=pair.append({'i': int(i),'j':int(j),column['Name']:avg},ignore_index=True)
                            
                    elif cell_pair_agg=='avg':
                        avg= np.mean(temp)
                        pair=pair.append({'i': int(i),'j':int(j),column['Name']:avg},ignore_index=True)
                        
                    elif cell_pair_agg=='best':
                        best= np.max(temp)
                        pair=pair.append({'i': int(i),'j':int(j),column['Name']:best},ignore_index=True)


                elif column['Type_col']=='Only Text':
                #Check for user input:    
                    temp=[]
                    if ('phonetic' in cf.get('Only Text'))==True:
                        score1=sm.phonetic(df.iloc[i][column['Name']],df.iloc[j][column['Name']])
                        temp.append(score1)
                    
                    if ('levenshtein' in cf.get('Only Text'))==True:
                        score2=sm.levenshtein(df.iloc[i][column['Name']],df.iloc[j][column['Name']])
                        temp.append(score2)
                        
                    if ('d_levenshtein' in cf.get('Only Text'))==True:
                        score3=sm.d_levenshtein(df.iloc[i][column['Name']],df.iloc[j][column['Name']])
                        temp.append(score3)
                    
                    if ('soft_tfidf' in cf.get('Only Text'))==True:
                        score4=sm.soft_tfidf(df.iloc[i][column['Name']],df.iloc[j][column['Name']])
                        temp.append(score4)
                        
                    if ('jaro_winkler' in cf.get('Only Text'))==True:
                        score5=sm.jaro_winkler(df.iloc[i][column['Name']],df.iloc[j][column['Name']])
                        temp.append(score5)
                        
                    if ('jaccard' in cf.get('Only Text'))==True:
                        score6=sm.jaccard(df.iloc[i][column['Name']],df.iloc[j][column['Name']])
                        temp.append(score6)
                        
                    if ('monge_eklan' in cf.get('Only Text'))==True:
                        score7=sm.monge_eklan(df.iloc[i][column['Name']],df.iloc[j][column['Name']])
                        temp.append(score7)
                        
                    if ('water' in cf.get('Only Text'))==True:
                        score8=sm.water(df.iloc[i][column['Name']],df.iloc[j][column['Name']])
                        temp.append(score8)
                    
                    #if ('cosine_string' in cf.get('Only Text'))==True:
                        #score9=sm.cosine_string(df.iloc[i][column['Name']],df.iloc[j][column['Name']])
                        #temp.append(score9)
                    
                    
                #Aggregation:
                    if cell_pair_agg=='wtavg':
                        if column['Name'] in cpw:
                            wt=cpw.get(column['Name'])
                            cell_wtavg= np.average(temp,weights=wt)
                            pair=pair.append({'i': int(i),'j':int(j),column['Name']:cell_wtavg},ignore_index=True)
                        else:
                            avg= np.mean(temp)
                            pair=pair.append({'i': int(i),'j':int(j),column['Name']:avg},ignore_index=True)
                            
                    elif cell_pair_agg=='avg':
                        avg= np.mean(temp)
                        pair=pair.append({'i': int(i),'j':int(j),column['Name']:avg},ignore_index=True)
                        
                    elif cell_pair_agg=='best':
                        best= np.max(temp)
                        pair=pair.append({'i': int(i),'j':int(j),column['Name']:best},ignore_index=True)

                elif column['Type_col']=='Text with Number & Special Character':
                #Check for user input:     
                    temp=[]
                    if ('levenshtein' in cf.get('Text with Number & Special Character'))==True:
                        score1=sm.levenshtein(df.iloc[i][column['Name']],df.iloc[j][column['Name']])
                        temp.append(score1)
                        
                    if ('d_levenshtein' in cf.get('Text with Number & Special Character'))==True:
                        score2=sm.d_levenshtein(df.iloc[i][column['Name']],df.iloc[j][column['Name']])
                        temp.append(score2)
                    
                    if ('soft_tfidf' in cf.get('Text with Number & Special Character'))==True:
                        score3=sm.soft_tfidf(df.iloc[i][column['Name']],df.iloc[j][column['Name']])
                        temp.append(score3)
                        
                    if ('jaro_winkler' in cf.get('Text with Number & Special Character'))==True:
                        score4=sm.jaro_winkler(df.iloc[i][column['Name']],df.iloc[j][column['Name']])
                        temp.append(score4)
                        
                    if ('jaccard' in cf.get('Text with Number & Special Character'))==True:
                        score5=sm.jaccard(df.iloc[i][column['Name']],df.iloc[j][column['Name']])
                        temp.append(score5)
                        
                    if ('monge_eklan' in cf.get('Text with Number & Special Character'))==True:
                        score6=sm.monge_eklan(df.iloc[i][column['Name']],df.iloc[j][column['Name']])
                        temp.append(score6)
                    
                    if ('cosine_string' in cf.get('Text with Number & Special Character'))==True:
                        score7=sm.cosine_string(df.iloc[i][column['Name']],df.iloc[j][column['Name']])
                        temp.append(score7)
                        
                    if ('water' in cf.get('Text with Number & Special Character'))==True:
                        score8=sm.water(df.iloc[i][column['Name']],df.iloc[j][column['Name']])
                        temp.append(score8)
                    
                #Aggregation:
                    if cell_pair_agg=='wtavg':
                        if column['Name'] in cpw:
                            wt=cpw.get(column['Name'])
                            cell_wtavg= np.average(temp,weights=wt)
                            pair=pair.append({'i': int(i),'j':int(j),column['Name']:cell_wtavg},ignore_index=True)
                        else:
                            avg= np.mean(temp)
                            pair=pair.append({'i': int(i),'j':int(j),column['Name']:avg},ignore_index=True)
                            
                    elif cell_pair_agg=='avg':
                        avg= np.mean(temp)
                        pair=pair.append({'i': int(i),'j':int(j),column['Name']:avg},ignore_index=True)
                        
                    elif cell_pair_agg=='best':
                        best= np.max(temp)
                        pair=pair.append({'i': int(i),'j':int(j),column['Name']:best},ignore_index=True)


    row_sim_score = pair.apply(lambda x: pd.Series(x.dropna().values))
    row_sim_score = row_sim_score.dropna()
    sim_score=0
    
    if row_pair_agg=='avg':
        row_sim_score['Avg'] = row_sim_score.iloc[:, 2:].mean(axis=1)
        sim_score = row_sim_score.pivot(index='i', columns='j', values='Avg')
    
    elif row_pair_agg=='wtavg':
        
        wt=[]
        for column in df:
            if column in w:
                wt.append(w.get(column))
            else:
                wt.append(1)
        
        data= row_sim_score.iloc[:, 2:]
        wtavg=pd.DataFrame(columns=['Wt_Avg'])

        for index, row in data.iterrows():
            wtavg=wtavg.append({'Wt_Avg':np.average(row, weights=wt)},ignore_index=True)

        row_sim_score =pd.concat([row_sim_score, wtavg], axis=1)
        sim_score = row_sim_score.pivot(index='i', columns='j', values='Wt_Avg')

    return(sim_score)

## || Test 1:

In [198]:
pairwise_matching('testpair.csv',cell_pair_agg='wtavg',row_pair_agg='avg')

j,0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,...,49.0,50.0,51.0,52.0,53.0,54.0,55.0,56.0,57.0,58.0
i,,,,,,,,,,,,,,,,,,,,,
0.0,1.0,0.494422,0.666730,0.652633,0.399692,0.679810,0.738225,0.406713,0.512120,0.527257,...,0.414655,0.452075,0.490335,0.240104,0.174164,0.509852,0.511279,0.656443,0.610185,0.505187
1.0,NaN,1.000000,0.418283,0.605112,0.696557,0.591131,0.595847,0.512241,0.586944,0.550481,...,0.546892,0.527487,0.383816,0.464072,0.372373,0.329189,0.330093,0.545366,0.561937,0.603571
2.0,NaN,NaN,1.000000,0.591401,0.364992,0.586049,0.577742,0.562963,0.458547,0.477334,...,0.363687,0.466955,0.638174,0.362067,0.238734,0.621484,0.454314,0.588599,0.591905,0.451345
3.0,NaN,NaN,NaN,1.000000,0.544253,0.783056,0.559687,0.534881,0.524271,0.664455,...,0.493882,0.562117,0.553314,0.370929,0.344231,0.503164,0.464703,0.738576,0.754693,0.647595
4.0,NaN,NaN,NaN,NaN,1.000000,0.526674,0.505824,0.596578,0.470705,0.502880,...,0.556586,0.437696,0.278972,0.577605,0.440745,0.299592,0.428960,0.480597,0.492942,0.534390
5.0,NaN,NaN,NaN,NaN,NaN,1.000000,0.565550,0.545145,0.570750,0.670230,...,0.512416,0.544040,0.513816,0.332168,0.287079,0.536894,0.498984,0.729557,0.722495,0.660331
6.0,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,0.331766,0.656143,0.421228,...,0.537693,0.311360,0.397596,0.384047,0.307826,0.449958,0.461426,0.584311,0.525053,0.453774
7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,0.299064,0.535038,...,0.389894,0.558898,0.486564,0.577780,0.354861,0.451516,0.449983,0.488059,0.515062,0.536939
8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,0.478624,...,0.529621,0.337799,0.508151,0.367952,0.329572,0.554320,0.380001,0.513344,0.522777,0.484758


## || Test 2:

In [197]:
pairwise_matching('pairwise.csv',cell_pair_agg='wtavg',row_pair_agg='avg')

j,0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,...,49.0,50.0,51.0,52.0,53.0,54.0,55.0,56.0,57.0,58.0
i,,,,,,,,,,,,,,,,,,,,,
0.0,1.0,0.382571,0.532699,0.439488,0.681190,0.533750,0.563029,0.598446,0.413979,0.410752,...,0.626530,0.588250,0.684144,0.366773,0.464126,0.401213,0.405856,0.409469,0.573900,0.418067
1.0,NaN,1.000000,0.467336,0.664375,0.306781,0.241162,0.388742,0.228590,0.428832,0.660477,...,0.475229,0.206796,0.308062,0.769556,0.531095,0.411796,0.394765,0.467391,0.371113,0.477978
2.0,NaN,NaN,1.000000,0.551375,0.539465,0.396098,0.444757,0.421963,0.668599,0.480889,...,0.552601,0.390569,0.563915,0.500205,0.565267,0.282500,0.468288,0.673126,0.449415,0.517541
3.0,NaN,NaN,NaN,1.000000,0.418856,0.245248,0.504123,0.319967,0.510964,0.570572,...,0.530598,0.301975,0.365259,0.627030,0.638959,0.344538,0.515819,0.567000,0.223211,0.582125
4.0,NaN,NaN,NaN,NaN,1.000000,0.621446,0.655645,0.613013,0.481490,0.329167,...,0.602770,0.605736,0.760977,0.314169,0.408051,0.544497,0.524666,0.506964,0.652575,0.393847
5.0,NaN,NaN,NaN,NaN,NaN,1.000000,0.497234,0.444127,0.336801,0.268338,...,0.424057,0.424438,0.578637,0.217431,0.204743,0.401305,0.517599,0.307545,0.582496,0.155810
6.0,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,0.497442,0.379787,0.332879,...,0.470719,0.476403,0.632613,0.379312,0.384668,0.579067,0.575391,0.481865,0.501172,0.341069
7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,0.442287,0.256362,...,0.511036,0.690031,0.588376,0.185497,0.356218,0.478068,0.341017,0.344263,0.503285,0.353517
8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,0.448167,...,0.514664,0.396851,0.476197,0.392364,0.510309,0.285371,0.403232,0.583308,0.367024,0.589836


## || Test 3:

In [196]:
pairwise_matching('honeyproduction.csv',cell_pair_agg='wtavg',row_pair_agg='avg')

j,0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,...,49.0,50.0,51.0,52.0,53.0,54.0,55.0,56.0,57.0,58.0
i,,,,,,,,,,,,,,,,,,,,,
0.0,1.0,0.635512,0.606250,0.484020,0.671640,0.535554,0.578326,0.601059,0.537830,0.671560,...,0.535661,0.446032,0.433812,0.439894,0.501156,0.452502,0.442222,0.434450,0.515767,0.448018
1.0,NaN,1.000000,0.725481,0.542181,0.573659,0.563105,0.667830,0.534634,0.605098,0.503771,...,0.588628,0.491964,0.406120,0.410744,0.435041,0.540906,0.438026,0.459347,0.473894,0.447463
2.0,NaN,NaN,1.000000,0.521185,0.565461,0.518344,0.672644,0.568373,0.575720,0.513313,...,0.521776,0.426526,0.461003,0.454277,0.434732,0.560298,0.433392,0.430673,0.490633,0.432644
3.0,NaN,NaN,NaN,1.000000,0.503667,0.540351,0.535535,0.438960,0.479592,0.381776,...,0.428088,0.390000,0.379261,0.264407,0.369790,0.422410,0.254793,0.295918,0.403484,0.310451
4.0,NaN,NaN,NaN,NaN,1.000000,0.533326,0.551037,0.592210,0.542296,0.569450,...,0.444262,0.440819,0.423130,0.402925,0.433051,0.434305,0.447580,0.417333,0.427017,0.473177
5.0,NaN,NaN,NaN,NaN,NaN,1.000000,0.524830,0.530477,0.521260,0.494321,...,0.419317,0.412932,0.428629,0.367987,0.412566,0.425536,0.349155,0.366165,0.442427,0.349477
6.0,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,0.543363,0.618445,0.510092,...,0.648153,0.452092,0.465538,0.465295,0.485275,0.582749,0.395006,0.432102,0.508729,0.417281
7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,0.537755,0.602131,...,0.454358,0.628892,0.443470,0.430240,0.552067,0.431177,0.382249,0.373323,0.512388,0.416413
8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,0.572279,...,0.464836,0.512962,0.643753,0.446924,0.494248,0.613354,0.373387,0.466959,0.393760,0.408924
